### Importing the required modules

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

2024-10-20 13:48:10.125082: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-20 13:48:14.399136: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-20 13:48:16.655057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 13:48:20.514828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 13:48:21.311812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 13:48:27.389305: I tensorflow/core/platform/cpu_feature_gu

### Generate sample data with demand quantities and some exogeneous features

In [2]:
# Generate sample data
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2022-12-31', freq='D')
demand = np.random.randint(50, 200, size=len(dates))
temperature = np.random.uniform(10, 30, size=len(dates))
is_weekend = (dates.dayofweek >= 5).astype(int)

data = pd.DataFrame({'demand': demand}, index=dates)
exog_features = pd.DataFrame({'temperature': temperature, 'is_weekend': is_weekend}, index=dates)

In [ ]:
data.head()

,demand
2020-01-01,152
2020-01-02,142
2020-01-03,64
2020-01-04,156
2020-01-05,121


In [4]:
exog_features.head()

,temperature,is_weekend
2020-01-01,12.355022,0
2020-01-02,21.530330,0
2020-01-03,15.481104,0
2020-01-04,21.083560,1
2020-01-05,23.028408,1


In [5]:
combined_data = pd.concat([data, exog_features], axis=1)
combined_data.head()

,demand,temperature,is_weekend
2020-01-01,152,12.355022,0
2020-01-02,142,21.530330,0
2020-01-03,64,15.481104,0
2020-01-04,156,21.083560,1
2020-01-05,121,23.028408,1


### Prepare the data

In [6]:
def prepare_data(data, look_back, forecast_horizon, features):
    X, y = [], []
    for i in range(len(data) - look_back - forecast_horizon + 1):
        X.append(data[i:(i + look_back), :])
        y.append(data[i + look_back:i + look_back + forecast_horizon, 0])
    return np.array(X), np.array(y)